# **Data Modelling and Evaluation**

## Objectives
- Answer business requirement 2 : Whether a leave is powdery-mildew 

## Inputs
* inputs/cherry_dataset/cherry-leaves/test
- inputs/cherry_dataset/cherry-leaves/train
* inputs/cherry_dataset/cherry-leaves/validation
* image shape embeddings.

## Outputs 
- Images distribution plot in train, validation, and test set.
- Image augmentation.
- Class indices to change prediction inference in labels.
- Machine learning model creation and training.
- Save the model.
- Learning curve plot for model performance.
- Model evaluation on pickle file.
- Prediction on the random image file.

# Import python packages

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread

# Setting up directory to the parent directory

In [4]:
cwd = os.getcwd()
cwd

'/Users/rana/Documents/artificial_intelligence/PP5-mildew-detection-in-cherry-leaves/jupyter_notebooks'

In [5]:
os.chdir('/Users/rana/Documents/artificial_intelligence/PP5-mildew-detection-in-cherry-leaves')
print('Parent directory set')

Parent directory set


In [6]:
current_dir = os.getcwd()
current_dir

'/Users/rana/Documents/artificial_intelligence/PP5-mildew-detection-in-cherry-leaves'

## Setting input directory

In [7]:
data_dir = 'inputs/cherry_dataset/cherry-leaves'
train_path = data_dir + '/train'
val_path = data_dir + '/validation'
test_path = data_dir + '/test'

## Setting Ouput directory


In [8]:
version = 'v1'
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(current_dir) and version in os.listdir(current_dir + '/outputs'):
    print('Old version is already available create a new version.')
    pass
else:
    os.makedirs(name=file_path)

Old version is already available create a new version.


## Setting labels

In [9]:
labels = os.listdir(train_path)

print(
    f"Project Labels: {labels}"
)

Project Labels: ['powdery_mildew', 'healthy']


## Import set image shape


In [10]:
import joblib
version = 'v1'
image_shape = joblib.load(filename=f"outputs/{version}/image_shape.pkl")
image_shape

(50, 50, 3)